<a href="https://colab.research.google.com/github/ami3001/FruitClassifier/blob/main/DeploymentVoila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 194kB 9.1MB/s 
     |████████████████████████████████| 12.8MB 233kB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [ ]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [ ]:
def download_file_from_google_drive(id,destination):
  URL = "https://docs.google.com/uc?export=download"
  session = requests.Session()
  response = session.get(URL, params = {'id' :id},stream = True)
  token = get_confirm_token(response)
  if token:
    params = {'id' : id, 'confirm':token}
    response = session.get(URL,params = params, stream = True)
    save_response_content(response, destination)
def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None
def save_response_content(response,destination):
  CHUNK_SIZE = 32768
  with open(destination, "wb") as f:
    for chunk in response.iter_content(CHUNK_SIZE):
      if chunk:
        f.write(chunk)

In [ ]:
file_id = '1-2vs8mpoPNJm9X5sndndIB_KBZHYn91C'
destination = './fruits_classifier.pk1'
download_file_from_google_drive(file_id,destination)

##Classifier zu Fruits 

Folgende Früchte werden erkannt:


*   Orange
*   Banana
*   Apple




In [ ]:
path = Path()
inference = load_learner(path/'fruits_classifier.pk1',cpu = True)

In [ ]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_p1 = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_p1.clear_output()
  with out_p1: display(img.to_thumb(128,128))
  pred,pred_idx,probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
  
btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your fruit!'),
              btn_upload,btn_run,out_p1,lbl_pred])